In [31]:
from IPython.core.magic import register_cell_magic
from black import format_str, FileMode

@register_cell_magic
def black_magic(line, cell):
    formatted_code = format_str(cell, mode=FileMode())
    get_ipython().set_next_input(formatted_code)

In [33]:
%%black_magic
import math,sys;
def example1(x,y):return math.sqrt(x+y)

In [ ]:
import math, sys


def example1(x, y):
    return math.sqrt(x + y)


In [35]:
%%black_magic
import pandas as pd
from skimpy import clean_columns
import pyarrow
from dotenv import load_dotenv
# Load environment variables from .env file
import os
from pyprojroot import here

In [ ]:
path = here("./")
os.chdir(path)
load_dotenv("env.txt")


In [ ]:
print(os.getenv('account_id'))

In [ ]:
import boto3

def list_object_versions(bucket_name, object_key):
    s3 = boto3.client("s3")
    response = s3.list_object_versions(Bucket=bucket_name, Prefix=object_key)
    
    for obj_version in response.get('Versions', []):
        print(f"VersionId: {obj_version['VersionId']}, LastModified: {obj_version['LastModified']}")
        
# Example usage
list_object_versions("your_bucket_name", "your_object_key")


In [8]:
#List buckets

870665956147


In [9]:
import boto3

s3 = boto3.client('s3')
response = s3.list_buckets()
buckets = [bucket['Name'] for bucket in response['Buckets']]
print("Bucket List: %s" % buckets)


Bucket List: ['sagemaker-studio-42jgegdgih', 'sagemaker-studio-870665956147-9g0yh5ygs3v', 'sagemaker-studio-870665956147-vpts40fdgw', 'sagemaker-studio-a7zxjm4whkd', 'sagemaker-studio-gsrv3qn27bw', 'sagemaker-studio-jjubn18fk5', 'sagemaker-studio-os4zjrtf8kk', 'sagemaker-studio-oyppxlu7ax', 'sagemaker-studio-tirqbmsca0b', 'sagemaker-studio-wzkj8ottow8', 'sagemaker-us-east-1-870665956147', 'sagemaker-us-east-2-870665956147']


In [11]:
bucket_name = 'sagemaker-studio-42jgegdgih'

In [12]:
import pandas as pd
import boto3
from io import StringIO

# Initialize the S3 client
s3 = boto3.client('s3')

# Create a simple DataFrame
df = pd.DataFrame({
    'col1': [1, 2, 3],
    'col2': ['a', 'b', 'c']
})

# Convert the DataFrame to a CSV string
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False)


# Upload the CSV string to S3
s3.put_object(Body=csv_buffer.getvalue(), Bucket=bucket_name, Key='df.csv')

{'ResponseMetadata': {'RequestId': 'N03J2C8WBG2XBCQ4',
  'HostId': 'ZU6XU+gL0JdOH4QeYpZ2eauju7FXk7mqA0b6wDaffxX1I85qDkbT0iEjbcjeRBpTWHfIwxFlk3I=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ZU6XU+gL0JdOH4QeYpZ2eauju7FXk7mqA0b6wDaffxX1I85qDkbT0iEjbcjeRBpTWHfIwxFlk3I=',
   'x-amz-request-id': 'N03J2C8WBG2XBCQ4',
   'date': 'Thu, 26 Oct 2023 08:25:04 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"c15df8f60a639bf3524486f845cd1459"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"c15df8f60a639bf3524486f845cd1459"',
 'ServerSideEncryption': 'AES256'}

In [15]:
import pandas as pd
import boto3
from io import StringIO

# Initialize the S3 client
s3 = boto3.client('s3')

# Fetch the file from S3
obj = s3.get_object(Bucket=bucket_name, Key='df.csv')

# Read the CSV content
csv_content = obj['Body'].read().decode('utf-8')

# Convert CSV content to DataFrame
csv_buffer = StringIO(csv_content)
df_download = pd.read_csv(csv_buffer)

# Now, df contains the same data as your original DataFrame
print(df_download)


   col1 col2
0     1    a
1     2    b
2     3    c


In [17]:
#Mflow example

In [22]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [23]:
# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and train a logistic regression model with increased max_iter
model = LogisticRegression(max_iter=1000)  # Increase the number of iterations
model.fit(X_train_scaled, y_train)  # Use scaled data

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

In [25]:
# Log parameters, metrics, and model using MLflow
with mlflow.start_run() as run:
    # Log parameters
    mlflow.log_param("solver", "lbfgs")
    mlflow.log_param("C", 1.0)

    # Log metric
    mlflow.log_metric("accuracy", accuracy)

    # Log model
    mlflow.sklearn.log_model(model, "model")

    # Register model to the model registry
    mlflow.register_model(
        f"runs:/{run.info.run_id}/model",
        "IrisModel"  # Name of the model in the registry
    )

/opt/conda/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'IrisModel' already exists. Creating a new version of this model...
2023/10/26 19:40:53 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: IrisModel, version 2
Created version '2' of model 'IrisModel'.
